In [44]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type, add_income_group, add_all_category_columns
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1500
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
add_race_eth(df) # Adds race_eth column
add_purpose_type(df) # Adds purpose_type column Combines loan_purpose and loan_type in the same way as the summary report
add_income_group(df) # Adds income_group column
add_all_category_columns(df) # Add '_c' categorical columns

ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
df = pd.merge(df, ts_name, on='respondent_id', how='outer')

Adding category column for  income_group
Adding category column for  loan_type
Adding category column for  loan_purpose
Adding category column for  race_eth


In [48]:
df.head()

,id,year,respondent_id,agency_code,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,action_type,msa_md,state_code,county_code,census_tract_number,applicant_ethnicity,co_applicant_ethnicity,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,...,applicant_income,purchaser_type,denial_reason_1,denial_reason_2,denial_reason_3,rate_spread,hoepa_status,lien_status,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msa_md_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator,race_eth,purpose_type,income_group,income_group_c,loan_type_c,loan_purpose_c,race_eth_c,respondent_name
0,1480794,2012,0000000151,5,1,1,3,1,105,3,1,41740,6,73,0098.04,1,5,5,NaN,NaN,NaN,NaN,8,NaN,NaN,...,239,0,NaN,NaN,NaN,NaN,2,1,NaN,1044,4773,25.96,75900,121.57,1597,1828,0,h,refinance,low,Low,Conventional,Refinancing,Hispanic,JUSTICE FEDERAL CREDIT UNION
1,1480794,2012,0000000151,5,1,1,3,1,105,3,1,41740,6,73,0098.04,1,5,5,NaN,NaN,NaN,NaN,8,NaN,NaN,...,239,0,NaN,NaN,NaN,NaN,2,1,NaN,1044,4773,25.96,75900,121.57,1597,1828,0,h,refinance,low,Low,Conventional,Refinancing,Hispanic,JUSTICE FEDERAL CREDIT UNION
2,1480794,2012,0000000151,5,1,1,3,1,105,3,1,41740,6,73,0098.04,1,5,5,NaN,NaN,NaN,NaN,8,NaN,NaN,...,239,0,NaN,NaN,NaN,NaN,2,1,NaN,1044,4773,25.96,75900,121.57,1597,1828,0,h,refinance,low,Low,Conventional,Refinancing,Hispanic,HABERSHAM BANK
3,1480794,2012,0000000151,5,1,1,3,1,105,3,1,41740,6,73,0098.04,1,5,5,NaN,NaN,NaN,NaN,8,NaN,NaN,...,239,0,NaN,NaN,NaN,NaN,2,1,NaN,1044,4773,25.96,75900,121.57,1597,1828,0,h,refinance,low,Low,Conventional,Refinancing,Hispanic,JUSTICE FEDERAL CREDIT UNION
4,1480794,2012,0000000151,5,1,1,3,1,105,3,1,41740,6,73,0098.04,1,5,5,NaN,NaN,NaN,NaN,8,NaN,NaN,...,239,0,NaN,NaN,NaN,NaN,2,1,NaN,1044,4773,25.96,75900,121.57,1597,1828,0,h,refinance,low,Low,Conventional,Refinancing,Hispanic,JUSTICE FEDERAL CREDIT UNION


In [46]:
# Function to create each section of the standard report. 
def standard_report_section(df, column_name, totalcount, totalamount):
    dfo = df.groupby(column_name)
    dfo = dfo['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    dfo['%count'] = np.round(100*dfo['count']/totalcount,1)
    dfo['%amount'] = np.round(100*dfo['amount']/totalamount,1)
    
    return dfo[['median','count','%count','amount','%amount']]

In [47]:
# Calling the standard_report_section
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()

# List Version
for col_name in ('purpose_type','race_eth','income_group', 'applicant_sex','occupancy'):
    frames.append(standard_report_section(df, col_name, totalcount, totalamount))

# Comprehension version
frames = [standard_report_section(df, column_name, totalcount, totalamount) for column_name in 
         ('purpose_type','race_eth','income_group','applicant_sex','occupancy')]
    
table = pd.concat(frames, keys=['Loan Purpose and Type', 
                               'Applicant Race/Eth', 
                               'Applicant Income',
                               'Applicant Sex',
                               'Occupancy'])

pieces = [table.ix[:,["median","count","%count","amount","%amount",]]]
table = pd.concat(pieces, axis=1)
table

median   count  %count  \
Loan Purpose and Type improvement               169   15157     1.9   
                      purchase-conventional     300  111411    13.7   
                      purchase-govt             319   73842     9.1   
                      refinance                 298  614648    75.4   
Applicant Race/Eth    aian                      280    2585     0.3   
                      as                        308   79999     9.8   
                      b                         271   13440     1.6   
                      h                         252   79690     9.8   
                      pi                        294    6348     0.8   
                      w                         305  476285    58.4   
Applicant Income      high                      769    3826     0.5   
                      low                       292  698621    85.7   
                      mid                       604    5015     0.6   
                      mod                       541   13882     1.7   
Applicant Sex         1.0                       312  534103    65.5   
                      2.0                       259  169785    20.8   
                      3.0                       298   59477     7.3   
                      4.0                       299   51693     6.3   
Occupancy             1.0                       307  700063    85.9   
                      2.0                       240  111401    13.7   
                      3.0                       800    3594     0.4   

                                                amount  %amount  
Loan Purpose and Type improvement              3343928      1.2  
                      purchase-conventional   41228964     15.1  
                      purchase-govt           25075218      9.2  
                      refinance              203787222     74.5  
Applicant Race/Eth    aian                      777383      0.3  
                      as                      26116565      9.6  
                      b                        3796581      1.4  
                      h                       21226260      7.8  
                      pi                       1909019      0.7  
                      w                      161302294     59.0  
Applicant Income      high                     4250966      1.6  
                      low                    217380466     79.5  
                      mid                      4245307      1.6  
                      mod                      9403461      3.4  
Applicant Sex         1.0                    183117648     67.0  
                      2.0                     47682222     17.4  
                      3.0                     20119568      7.4  
                      4.0                     22515894      8.2  
Occupancy             1.0                    235073293     86.0  
                      2.0                     30946476     11.3  
                      3.0                      7415563      2.7